<DIV ALIGN=CENTER>

# Introduction to Cassandra
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

In this Notebook, we introduce [Cassandra][cdb], one of the more popular
NoSQL databases. [Cassandra][wcdb] is a column-oriented database that
provides high-availability. Cassandra utilizes Cassandra Query Language
(CQL) to interact with a Cassandra database. Since Cassandra utilizes
tables and schemas like a relational database, CQL will also be familiar
to anyone who has been exposed to SQL. Given the scale and scope of
Cassandra, this Notebook will focus on the simple tasks of connecting to
a Cassandra instance from Python, and inserting, updating, selecting,
and deleting records from a Cassandra database.

-----
[cdb]: http://cassandra.apache.org
[wcdb]: https://en.wikipedia.org/wiki/Apache_Cassandra
[mdbd]: https://docs.mongodb.org/manual/

## Python with MongoDB

To use Python to interact with MongoDB, we need to use a suitable Python
library. The recommended Python library is [_pymongo_][pymdb], which
provides support for establishing a connection between a Python program
and a MongoDB server as well as support tools for working with MongoDB. 

We have already installed _pymongo_ in the course Docker container;
however, you can easily install is by using `pip`, for example to
install _pymongo_ for use with Python3 for the current user, we can
execute:

```console
pip3 install pymongo --user
```

Once this library is installed, we can import the MongoDB client to
establish a connection and retrieve data and MongoDB information.

```python
from pymongo import MongoClient
```

-----

[pymdb]: http://api.mongodb.org/python/current/

## Python and Cassandra

To use Python to interact with Cassandra, we need to use a suitable
Python library. The recommended Python library is the
[_cassandra-driver_][py2cass], which provides support for establishing a
connection between a Python program and a Cassabdra database as well as
methods for sending CQL queries to Cassandra and to process the results. 

We have already installed _cassandra-driver_ in the course Docker
container; however, you can easily install is by using `pip`, for
example to install _cassandra-driver_ for use with Python3 for the
current user, we can execute:

```bash
pip3 install cassandra-driver --user
```

Once this library is installed, we can import necessary classes from the
Cassandra client to establish a connection and retrieve data. In
particular this will include `cluster`, which will allow us to establish
a connection to a remote database.

```python
from cassandra.cluster import Cluster
```

-----

[py2cass]: https://github.com/datastax/python-driver

In [1]:
from cassandra.cluster import Cluster

-----

## Remote Cassandra Server

To connect to a remote Cassandra server, for instance by using the course
cluster system, we simply need the IP address for the server and the
port number on which the Cassandra daemon is listening. For this course,
Notebooks running on the course JupyterHub Server can access a Cassandra
server, hosted by Micrsoft Azure, on `40.124.12.119` on the default port of `9042`:


```python
cluster = Cluster(['40.124.12.119'])
```

-----

In [2]:
cassandra_ips = ['40.124.12.119']

# Establish a connection to Cassandra

# The Policy is necessary to allow Cassandra to run on Azure.
from cassandra.policies import WhiteListRoundRobinPolicy
pcy = WhiteListRoundRobinPolicy(cassandra_ips)

# Create Connection
cluster = Cluster(contact_points=cassandra_ips, load_balancing_policy=pcy)
session = cluster.connect()

print('Cluster Name: {0}'.format(cluster.metadata.cluster_name))
for host in cluster.metadata.all_hosts():
    print('{0}: Host: {1} in {2}'\
          .format(host.datacenter, host.address, host.rack))

Cluster Name: lcdm
datacenter1: Host: 10.3.0.5 in rack1
datacenter1: Host: 40.124.12.119 in rack1


-----

## Cassandra Keyspace

Cassandra uses a _keyspace_ to provide a namespace that can organize
related data. To provide distinct environments for each student, each
student will create their own keyspace in the shared Cassandra cluster
by using their netids. The following code cell reads the netid from the
docker container filesystem.

-----

In [3]:
# Filename containing user's netid
fname = '/home/data_scientist/users.txt'
with open(fname, 'r') as fin:
    netid = fin.readline().rstrip()

# We will delete our working directory if it exists before recreating.
ks_name = '{0}'.format(netid)

### Creating a Keyspace

In this course, we separate each student's work by using a separate
keyspace for each  student. To do this, the following code cell uses
Python code to first drop the user's keyspace if it exists, and to then
create the user's keyspace. This effectively deletes all data added to
the keyspace from this Notebook. Alternatively, we could employ CQL
(Cassandra Query Language) to perform the same steps:

```python
# CQL to drop named keyspace
drop_ks = '''
DROP KEYSPACE IF EXISTS {0} ;
'''

# CQL to create named keyspace (We need to escape curly braces)
create_ks = '''
CREATE KEYSPACE {0} \
    WITH REPLICATION = {{'class': 'SimpleStrategy', 'replication_factor': 1}} ;
'''

# Execute CQL for Keyspaces
session.execute(drop_ks.format(ks_name))
session.execute(create_ks.format(ks_name))

# Display Keyspace names
keys = [val for val in sorted(cluster.metadata.keyspaces.keys()) if 'system' not in val]
for ks in keys:
    print(ks)
```

In [4]:
# Boilerplate code we need to create keyspace from within Python
# Altenaitve is to create keyspace from CQL.

from cassandra.query import dict_factory
session.row_factory = dict_factory

from cassandra.cqlengine import connection, management
connection.set_session(session)

# Explicitly set session hosts, this removes annoying warnings.
connection.session.hosts = cassandra_ips

# Drop Keyspace if it exists
if ks_name in cluster.metadata.keyspaces:
    management.drop_keyspace(ks_name)

# Create Keyspace
management.create_keyspace_simple(ks_name, 1)

# Set keyspace for this session
# Note: If keyspace exists in Cassandra instance, this is only line we need.
session.set_keyspace(ks_name)

# Display all non-system keyspaces.
# Do not change to a different keyspace!

keys = [val for val in sorted(cluster.metadata.keyspaces.keys()) if 'system' not in val]
for ks in keys:
    print(ks)

print('\nCQL Query to recreate this keyspace:')
print(40*'-')
print(cluster.metadata.keyspaces[ks_name].export_as_string())

/home/data_scientist/.local/lib/python3.5/site-packages/cassandra/cqlengine/management.py:431: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


bigdog
info490

CQL Query to recreate this keyspace:
----------------------------------------
CREATE KEYSPACE bigdog WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}  AND durable_writes = true;


----

### Schema Creation

Cassandra uses tables, which have well defined schemas, to hold data.
Anyone familiar with relational databases and SQL will be able to
quickly begin using [CQL][cql]. In the following code cell, we first
drop the table if it exists. We do this to ensure a clean slate before
we create our new schema and insert data. Next, we write a `CREATE`
statement that build a `students` table to hold several attributes.
While this schema is simple, Cassandra allows for [complex schema][cs]
to be created by using CQL.

One item of importance is the creation of the _Primary Key_. A primary
key can be simple, using only a single column. Or it can be
[compound][cc], including a [partition key][cp] to specify what nodes
contain the data. In our case, we do not need partitions since our data
are all hosted by the same node and there is no redundancy.


-----

[cql]: http://docs.datastax.com/en/cql/3.3/cql/cqlIntro.html
[cs]: http://docs.datastax.com/en/cql/3.3/cql/cql_using/useCreateTableTOC.html
[cc]: http://docs.datastax.com/en/cql/3.3/cql/cql_using/useCompoundPrimaryKeyConcept.html
[cp]: http://docs.datastax.com/en/cql/3.3/cql/cql_using/useCompositePartitionKeyConcept.html

In [5]:
drop_schema = '''
DROP TABLE IF EXISTS students ;
'''

create_schema = '''
CREATE TABLE students (
    id int,
    fname text,
    lname text,
    company text,
    hire_date timestamp,
    PRIMARY KEY(lname, id)
);
'''
print('\nDeleting Table')
print(40*'-')
d_result = session.execute(drop_schema)
print(d_result.response_future)

print('\nCreating Table')
print(40*'-')
c_result = session.execute(create_schema)
print(c_result.response_future)


Deleting Table
----------------------------------------
<ResponseFuture: query='<SimpleStatement query="
DROP TABLE IF EXISTS students ;
", consistency=Not Set>' request_id=20 result=None exception=None host=40.124.12.119>

Creating Table
----------------------------------------
<ResponseFuture: query='<SimpleStatement query="
CREATE TABLE students (
    id int,
    fname text,
    lname text,
    company text,
    hire_date timestamp,
    PRIMARY KEY(lname, id)
);
", consistency=Not Set>' request_id=21 result=None exception=None host=40.124.12.119>


-----

## Adding Data

Given a collection, we can easily add new _records_ (or rows) to our
Cassandra database by creating a CQL `INSERT` statement. As was the case
with a relational database tables, a Cassandra table has a defined
schema, which means our insert statements will be similar to a SQL
insert statement. In the following code cell, we first create a
`student` dictionary, followed by an insert CQL statement. While we could
hard code the student data into the insert statement, we are using
parameters to indicate the values will be supplied to the actual CQL
`execute` statement. The syntax of these parameters is straightforward.
For example, the parameter indicated by the `%(id)s` sequence means we
will pass in the value as a dictionary, with the value corresponding to
the `id` key being used in the query replacing the parameter in the CQL
query. The `s` character simply means the data is transferred as a
string.


-----

In [6]:
import uuid
from datetime import datetime

student = {'id': 1,
           'fname': 'Jane',
           'lname': 'Doe',
           'company': 'bdg surf shop',
           'hire_date': datetime.now()}

insert_one = '''
INSERT INTO students (id, fname, lname, company, hire_date) 
VALUES (%(id)s, %(fname)s, %(lname)s, %(company)s, %(hire_date)s) ;
'''

io_result = session.execute(insert_one, student) 

print(io_result.response_future)

<ResponseFuture: query='<SimpleStatement query="
INSERT INTO students (id, fname, lname, company, hire_date) 
VALUES (%(id)s, %(fname)s, %(lname)s, %(company)s, %(hire_date)s) ;
", consistency=Not Set>' request_id=31 result=None exception=None host=40.124.12.119>


-----

We can also insert data by using a prepared statement, which provides
better performance since the database can _compile_ the CQL statement
and subsequently insert specific values in for the named placeholders.
We demonstrate this technique in the following two code cells. First, we
define the prepared statement, which in this case inserts data into the
`students` table. The values to insert are parameterized (e.g., as `:id`
and `:company`), which tells the database these values will be passed in
whenever the CQL statement is actually executed. The statement is sent
to the database to be prepared by calling the `prepare` method with the
CQL.

In the second cell, we process a dictionary of data to create two new
records, and we pass the prepared statement into the `execute`
statement, followed by the values. We also pass the values in for a
third record directly to demonstrate the flexibility of this approach.

-----

In [7]:
insert_many = '''
INSERT INTO students (id, fname, lname, company, hire_date) 
VALUES (:id, :fname, :lname, :company, :hire_date) ;
'''

im_prepared = session.prepare(insert_many)

In [8]:
new_students = [
    {'fname': 'Mike',
     'lname': 'Simone',
     'company': 'Del Ray Enterprises'},
    {'fname': 'Clair',
     'lname': 'Hwu',
     'company': 'Hoboken Surfware Incorporated'}]

# Insert multiple entires
for idx, student in enumerate(new_students):
    im_result = session.execute(im_prepared, ((idx + 1), 
                                              student['fname'], student['lname'],
                                              student['company'], datetime.now()))
    
# Insert one using direct value substitution
im_result = session.execute(im_prepared, (3, 'John', 'Doe', "Bigdog's Surf Shop",
                                          datetime.now()))

-----

## Retrieving Data

CQL is very similar to SQL in many respects, which makes many
operations, such as querying a Cassandra database, very familiar. For
example, to query (or retrieve) records, we simply issue a select
statement. As was the case with SQL, a simple CQL select statement has a
`SELECT` clause, a `FROM` clause, followed by an optional `WHERE`
clause. The `SELECT` clause indicates what should be retrieved from the
database, and can be aggregate functions, such as `COUNT` or attribute
values. The `FROM` clause specifies the location of the records and can
include join conditions, and the `WHERE` clause filters the records.

In the following code cells, we demonstrate three different queries.
First, we return a count of the records by using a `SELECT COUNT(*)`
query on the `students` table. Second, we query a select group of
attributes from the same table. Finally, we use a prepared statement
to select specific columns from the `students` table with a specific
condition, in this case we specify the record `id` attribute. In each
case, the result set from the query contains the data of interest,
either the count of records, or the selected columns, which are accessed
as a dictionary from the result set cursor.

-----

In [9]:
# Query for the record count
qr = session.execute('SELECT COUNT(*) FROM students')

# Dump the query and result set.
print(qr.response_future)

print(40*'-')
print('Employee count = {0}'.format(qr[0]['count']))

<ResponseFuture: query='<SimpleStatement query="SELECT COUNT(*) FROM students", consistency=Not Set>' request_id=36 result=[{'count': 4}] exception=None host=40.124.12.119>
----------------------------------------
Employee count = 4


In [10]:
query = '''
SELECT fname, lname, company, hire_date FROM  students ;
'''

query_result = session.execute(query)

# Display query results
print(40*'-')
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))
    print(40*'-')

----------------------------------------
Name: Jane Doe
Company: bdg surf shop
Hired on 2016-04-16 17:49:42.323999
----------------------------------------
Name: John Doe
Company: Bigdog's Surf Shop
Hired on 2016-04-16 17:49:42.528000
----------------------------------------
Name: Clair Hwu
Company: Hoboken Surfware Incorporated
Hired on 2016-04-16 17:49:42.482000
----------------------------------------
Name: Mike Simone
Company: Del Ray Enterprises
Hired on 2016-04-16 17:49:42.438999
----------------------------------------


In [11]:
# We can easily query on Primary Keys
query = '''
SELECT fname, lname, company, hire_date 
FROM  students 
WHERE lname = %(ln)s ;
'''

query_result = session.execute(query, {'ln': 'Doe'})

# Display query results
print(40*'-')
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))
    print(40*'-')

----------------------------------------
Name: Jane Doe
Company: bdg surf shop
Hired on 2016-04-16 17:49:42.323999
----------------------------------------
Name: John Doe
Company: Bigdog's Surf Shop
Hired on 2016-04-16 17:49:42.528000
----------------------------------------


-----

## Modifying Data

Once records have been added to a Cassandra database, we can issue CQL
update statements to update or modify attributes in one or more records.
The format of an update statement is simple. We start by specifying the
`UPDATE` clause, next comes the `SET` clause where indicate what
attribute should be modified and the new value, after which comes a
`WHERE` clause that actually selects the record or records of interest.
In the following code cells, we first generate new temporary data to
demonstrate updating (and subsequently deleting) data from a Cassandra
database. 

-----

In [12]:
temp_students = [{'id': 11, 'fname': 'Petr', 'lname': 'Dow', 'company': 'bdg surf shop'},
                 {'id': 12, 'fname': 'Paul', 'lname': 'Dow', 'company': 'bdg surf shop'},
                 {'id': 13, 'fname': 'Mary', 'lname': 'Dow', 'company': 'bdg surf shop'},
                 {'id': 14, 'fname': 'Arthur', 'lname': 'Dow', 'company': 'bdg surf shop'}]

# Insert multiple entires
for student in temp_students:
    im_result = session.execute(im_prepared, (student['id'], 
                                              student['fname'], student['lname'],
                                              student['company'], datetime.now()))

# Count how many students
qr = session.execute('SELECT COUNT(*) FROM students')

print(40*'-')
print('Employee count = {0}'.format(qr[0]['count']))

----------------------------------------
Employee count = 8


In [13]:
# Update one record to fix the typo in the name.
ud_stmt = '''
UPDATE students
  SET fname = 'Peter'
  WHERE lname = 'Dow' AND id = 11 ;
'''

ur = session.execute(ud_stmt)

# Display Updated Row
query_result = session.execute(query, {'ln': 'Dow'})

# Display query results
for row in query_result:
    if row['fname'] == 'Peter':
        print('Name: {0} {1}'.format(row['fname'], row['lname']))
        print('Company: {0}'.format(row['company']))
        print('Hired on {0}'.format(row['hire_date']))    

Name: Peter Dow
Company: bdg surf shop
Hired on 2016-04-16 17:49:42.808000


-----

To update multiple rows, we can either generate multiple update
statements, perhaps by iterating over values used in a `WHERE` clause,
and execute multiple individual update statements, or, alternatively, we
can issue a single `UPDATE` statement where multiple rows are directly
selected in the `WHERE` clause. The latter approach generally provides
the best performance, and is recommended. In addition, this mechanism
enables multiple approaches to selecting the target rows. For example,
the rows might be selected by employing a range query or by providing a
set of allowed values for the target attribute. The latter technique is
demonstrated in the following code cell, where the `id` attribute is
selected to be in the set of values `(11, 12, 13, 14)`.

-----

In [14]:
# Update company name for multiple records.
ud_stmt = '''
UPDATE students
  SET company = 'Bigdogs Surf Shop'
  WHERE lname = 'Dow' AND id IN (11, 12, 13, 14) ;
'''

ur = session.execute(ud_stmt)

# Display Updated Rows
query_result = session.execute(query, {'ln': 'Dow'})

# Display query results
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))    
    print(40*'-')

Name: Peter Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.808000
----------------------------------------
Name: Paul Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.854000
----------------------------------------
Name: Mary Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.913000
----------------------------------------
Name: Arthur Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.960999
----------------------------------------


-----

## Deleting Data

To delete data, we construct a suitable CQL statement that leverages the
`DELETE` clause to delete data identified in the `WHERE` clause. For
example, int he following code cell we delete all rows where the last
name is `Dow` and the record id is `11`. We execute the query and
execute the previous query to demonstrate the target record was removed
form the database. In the second code cell, we delete multiple nodes by
using a prepared statement that allows us to specify values at run time. 

-----

In [15]:
del_stmt = '''
DELETE FROM students
WHERE lname = 'Dow' AND id = 11 ;
'''

dr = session.execute(del_stmt)
                     
# Display Updated Rows
query_result = session.execute(query, {'ln': 'Dow'})

# Display query results
for row in query_result:
    print('Name: {0} {1}'.format(row['fname'], row['lname']))
    print('Company: {0}'.format(row['company']))
    print('Hired on {0}'.format(row['hire_date']))    
    print(40*'-')


Name: Paul Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.854000
----------------------------------------
Name: Mary Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.913000
----------------------------------------
Name: Arthur Dow
Company: Bigdogs Surf Shop
Hired on 2016-04-16 17:49:42.960999
----------------------------------------


In [16]:
# Count how many students
qr = session.execute("SELECT COUNT(*) FROM students WHERE lname = 'Dow'")

print(40*'-')
print('Pre-Delete')
print('Employee (Dow) count = {0}'.format(qr[0]['count']))                

del_stmt = '''
DELETE FROM students
WHERE lname = 'Dow' AND id = %(id)s ;
'''

for id in [12, 13, 14]:
    dr = session.execute(del_stmt, {'id': id})
                     
# Count how many students
qr = session.execute("SELECT COUNT(*) FROM students WHERE lname = 'Dow'")

print(40*'-')
print('Post-Delete')
print('Employee (Dow) count = {0}'.format(qr[0]['count']))                

----------------------------------------
Pre-Delete
Employee (Dow) count = 3
----------------------------------------
Post-Delete
Employee (Dow) count = 0


-----

## Keyspace Dump

Clearly CQL is a powerful tool, and can be used to perform all
operations one might need to do against a Cassandra instance.
Constructing a CQL query can, therefore, be difficult. Fortunately,
Cassandra allows one to easily construct the CQL query required to
recreate a keyspace. Not only this is instructional, but it also is a
great tool to provide a schema dump that can be used in emergency
situations to reconstruct a Cassandra database. To generate this CQL
dump, one simply calls the `export_as_string` method on the named
keyspace, as shown below.


-----

In [17]:
print('CQL Query to recreate this keyspace:')
print(40*'-')
print(cluster.metadata.keyspaces[ks_name].export_as_string())

CQL Query to recreate this keyspace:
----------------------------------------
CREATE KEYSPACE bigdog WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}  AND durable_writes = true;

CREATE TABLE bigdog.students (
    lname text,
    id int,
    company text,
    fname text,
    hire_date timestamp,
    PRIMARY KEY (lname, id)
) WITH CLUSTERING ORDER BY (id ASC)
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '64', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND dclocal_read_repair_chance = 0.1
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interva

-----
### Student Activity

In the preceding cells, we introduced the Cassandra database and the
Python Cassandra database driver. Now that you have run the Notebook, go
back and make the following changes to see how the results change.

1. Try performing a CQL query that uses a range `WHERE` clause.
2. Create your own table and load data into it using prepared statements.
3. Create two tables, the data you should be related across the tables
to enable joins. For example, a friend table and a student table (since
fellow student may or may not be your fiends). insert some fake data
and query the data to find students who are friends by using a CQL join.

-----